<a href="https://colab.research.google.com/github/TobyChen320/Awesome-Profile-README-templates/blob/master/Revenue_Cycle_Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is an attempt to reformat a raw remit text file to the format we normally see when we pass the file into "HIPAA to Excel"
import pandas as pd

df = pd.read_csv("/content/NYCBS_HOSP_REMIT_1199_NATIONAL_BEN_FUND_03273OU101_PST.DAT", delimiter ='\t')
df.to_excel('Remit_Test.xlsx', index =False)
df_excel = pd.read_excel('Remit_Test.xlsx')
df_excel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv("WO.csv")
unique_counts = df['Comments'].value_counts().reset_index()
unique_counts.columns = ['Unique_values', 'Count']
unique_counts.to_csv('100%_WO_Counts.csv', index =False)

In [ ]:
# This block of code will append your data in order. You just need to change the sheet name and column names.
import os
import pandas as pd
directory = 'EKG'

excel_files = sorted([filename for filename in os.listdir(directory) if filename.startswith('EKG')])

combined_data_sheet1 = pd.DataFrame()
combined_data_sheet2 = pd.DataFrame()

for filename in excel_files:
  file_path = os.path.join(directory, filename)
  xls = pd.ExcelFile(file_path)
  for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name = sheet_name)
    if sheet_name == 'CARD_EKG_NM_C_R_Statuses':
      df['Source_File'] = filename
      combined_data_sheet1 = combined_data_sheet1.append(df, ignore_index = True)
    elif sheet_name == 'CARD_EKG_NM_All_Statuses':
      df['Source_File'] = filename
      combined_data_sheet2 = combined_data_sheet2.append(df, ignore_index = True)


with pd.ExcelWriter('EKG_Combined.xlsx', engine = 'openpyxl') as writer:
  combined_data_sheet1.to_excel(writer, sheet_name = 'CARD_EKG_NM_C_R_Statuses', index = False)
  combined_data_sheet2.to_excel(writer, sheet_name = 'CARD_EKG_NM_All_Statuses')

In [ ]:
# This will sort multiple column ranges for you in excel.
import pandas as pd

df = pd.read_excel('NEW_Charge_Test.xlsx')
df_sorted = df.sort_values(by=['UnitNumber',	'AccountNumber', 'ServiceDateTime'])
df_sorted.to_excel('sorted_new_charge.xlsx', index = False)

In [ ]:
# Appends the texts inside each file into 1 file.
import fileinput
import glob

file_list = glob.glob("*.BAK")

with open('Claims2022.txt', 'w') as file:
    input_lines = fileinput.input(file_list, encoding='cp1252')
    file.writelines(input_lines)

In [ ]:
# Change UCRN based on Row Update Time
import pandas as pd

df = pd.read_excel("Dyann 99202_march 2024 to march 5 2025 visits Nate file.xlsx")

# Ensure RowUpdateDateTime is a datetime type
# df["RowUpdateDateTime (BarBillsUB92Claims)"] = pd.to_datetime(df["RowUpdateDateTime (BarBillsUB92Claims)"])

# Get the most recent row for each AccountNumber
result = df.loc[df.groupby("AccountNumber")["AccountQueryID"].idxmax()]

result.to_excel("Test.xlsx", index=False)

In [ ]:
# Adhoc Pneumonia python format code.
import pandas as pd

# Create DataFrame
df = pd.read_csv("Pneumonia.csv")

grouped = df.groupby(['Account Number', 'Admit Date Time', 'Discharge Date Time', 'Final Drg'])['Diagnosis'].apply(list).reset_index()

# Create a new DataFrame with the desired format
max_diagnoses = grouped['Diagnosis'].apply(len).max()
diagnosis_columns = [f'Diagnosis {i+1}' for i in range(max_diagnoses)]

# Expand the list of diagnoses into separate columns
expanded_diagnoses = pd.DataFrame(grouped['Diagnosis'].tolist(), columns=diagnosis_columns)

# Combine the expanded diagnoses with the original grouped DataFrame
result = pd.concat([grouped.drop(columns=['Diagnosis']), expanded_diagnoses], axis=1)

# Reorder columns to match the desired output
column_order = ['Account Number', 'Admit Date Time'] + diagnosis_columns + ['Discharge Date Time', 'Final Drg']
result = result[column_order]

result.to_csv("Pneumonia 4th Quarter Not In DRG.csv", index=False)

This excel formula combines 2 column's values with a comma seperating them only if there are 2 values. Otherwise just returns the one. (Currently meant for TPA Charge Detail File)

=IF(AND(J2<>"", K2<>""), J2 & ", " & K2, J2 & K2)

This formula takes each value in the cell and wraps it in single apostrophes while seperating them with a comma

="'" & TEXTJOIN("', '", TRUE, A1:A2907) & "'"

In [ ]:
# Removes rows with null values for service to and service from date fields
# Working Claim Summary File
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_csv("Claim Summary File Final.csv")

# Ensure 'BillNumberID (BarBills)' and 'Type Of Bill' are numeric for comparison
df['BillNumberID (BarBills)'] = pd.to_numeric(df['BillNumberID (BarBills)'], errors='coerce')

# Step 1: Handle null values in 'BillNumberID (BarBills)' by replacing them with a very low value
df['BillNumberID (BarBills)'].fillna(-float('inf'), inplace=True)

# Convert 'Health Plan ID Number' to numeric and integer type to avoid decimal issues
df['Health Plan ID Number'] = pd.to_numeric(df['Health Plan ID Number'], errors='coerce').astype('Int64')

# Identify the row with the maximum 'BillNumberID (BarBills)' for each 'Account Number'
max_qid_rows = df.loc[df.groupby('Encounter ID')['BillNumberID (BarBills)'].idxmax()]

# Restore null values in 'BillNumberID (BarBills)' where applicable
max_qid_rows.loc[max_qid_rows['BillNumberID (BarBills)'] == -float('inf'), 'BillNumberID (BarBills)'] = None

# Step 2: Consolidate all fields for each 'Account Number'
# Exclude the fields that should not be consolidated ('Type Of Bill', 'Acct Num QID', etc.)
fields_to_exclude = ['BillNumberID (BarBills)']
consolidated_data = df.groupby('Encounter ID').agg(
    lambda x: ', '.join(sorted(x.dropna().astype(str).unique()))
)

# Step 3: Replace consolidated values for excluded fields with values from max_qid_rows
for field in fields_to_exclude:
    consolidated_data[field] = max_qid_rows.set_index('Encounter ID')[field]

# Step 4: Merge the consolidated fields into a single DataFrame
result_df = max_qid_rows[['Encounter ID']].merge(
    consolidated_data,
    on='Encounter ID',
    how='left'
)

# Reset the index for clarity
result_df = result_df.reset_index(drop=True)

# Step 5: Filter the DataFrame to only include the desired fields
desired_fields = [
    'Facility Name', 'Encounter ID', 'Patient Type', 'Type of Bill',
    'Type of Admission', 'Service From Date', 'Service To Date', 'Payor Plan Name',
    'Health Plan ID Number', 'Financial Class ID', 'Date of Birth', 'Sex',
    'Discharge status', 'Total Charges', 'Final Drg', 'MS DRG', 'Diagnosis Codes', 'Procedure Codes'
]

result_df = result_df[desired_fields]

# Step 6: Drop rows with blank Service From Date or Service To Date
result_df = result_df[
    (result_df['Service From Date'] != '') &
    (result_df['Service To Date'] != '')
]

# Save the resulting DataFrame to a new CSV file
result_df.to_csv("Claim Summary Month Sample.csv", index=False)

<ipython-input-3-39f240a4c187>:5: DtypeWarning: Columns (4,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Claim Summary File Final.csv")
<ipython-input-3-39f240a4c187>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['BillNumberID (BarBills)'].fillna(-float('inf'), inplace=True)


In [ ]:
# OLD
# Working Claim Summary File
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_csv("Claim Summary File Final.csv")

# Ensure 'BillNumberID (BarBills)' and 'Type Of Bill' are numeric for comparison
df['BillNumberID (BarBills)'] = pd.to_numeric(df['BillNumberID (BarBills)'], errors='coerce')

# Step 1: Handle null values in 'BillNumberID (BarBills)' by replacing them with a very low value
df['BillNumberID (BarBills)'].fillna(-float('inf'), inplace=True)

# Convert 'Health Plan ID Number' to numeric and integer type to avoid decimal issues
df['Health Plan ID Number'] = pd.to_numeric(df['Health Plan ID Number'], errors='coerce').astype('Int64')

# Identify the row with the maximum 'BillNumberID (BarBills)' for each 'Account Number'
max_qid_rows = df.loc[df.groupby('Encounter ID')['BillNumberID (BarBills)'].idxmax()]

# Restore null values in 'BillNumberID (BarBills)' where applicable
max_qid_rows.loc[max_qid_rows['BillNumberID (BarBills)'] == -float('inf'), 'BillNumberID (BarBills)'] = None

# Step 2: Consolidate all fields for each 'Account Number'
# Exclude the fields that should not be consolidated ('Type Of Bill', 'Acct Num QID', etc.)
fields_to_exclude = ['BillNumberID (BarBills)']
consolidated_data = df.groupby('Encounter ID').agg(
    lambda x: ', '.join(sorted(x.dropna().astype(str).unique()))
)

# Step 3: Replace consolidated values for excluded fields with values from max_qid_rows
for field in fields_to_exclude:
    consolidated_data[field] = max_qid_rows.set_index('Encounter ID')[field]

# Step 4: Merge the consolidated fields into a single DataFrame
result_df = max_qid_rows[['Encounter ID']].merge(
    consolidated_data,
    on='Encounter ID',
    how='left'
)

# Reset the index for clarity
result_df = result_df.reset_index(drop=True)

# Step 5: Filter the DataFrame to only include the desired fields
desired_fields = [
    'Facility Name', 'Encounter ID', 'Patient Type', 'Type of Bill',
    'Type of Admission', 'Service From Date', 'Service To Date', 'Payor Plan Name',
    'Health Plan ID Number', 'Financial Class ID', 'Date of Birth', 'Sex',
    'Discharge status', 'Total Charges', 'Final Drg', 'MS DRG', 'Diagnosis Codes', 'Procedure Codes'
]

result_df = result_df[desired_fields]

# Save the resulting DataFrame to a new CSV file
result_df.to_csv("Claim Summary Month Sample.csv", index=False)

print("Consolidation complete! Saved as 'Claim Summary Month Sample.csv'.")

KeyError: 'BillNumberID (BarBills)'

In [ ]:
# Charge Detail File Step 1

import pandas as pd

# Load the data
df = pd.read_csv("Charge Detail File.csv")

# Convert 'EffectiveDateTime' to datetime format
df['EffectiveDateTime (DBarProcAltCodeEffectDates)'] = pd.to_datetime(df['EffectiveDateTime (DBarProcAltCodeEffectDates)'])

# Separate rows where 'EffectiveDateTime' is NaT before filtering
nat_rows = df[df['EffectiveDateTime (DBarProcAltCodeEffectDates)'].isna()]

# Keep rows with the latest effective date for each 'ProcedureID (DBarProcedures)'
filtered_df = df[
    df['EffectiveDateTime (DBarProcAltCodeEffectDates)'].notna() &
    (df.groupby('CDM Item Code')['EffectiveDateTime (DBarProcAltCodeEffectDates)']
     .transform('max') == df['EffectiveDateTime (DBarProcAltCodeEffectDates)'])
]

# Combine the filtered rows with those that had NaT in 'EffectiveDateTime'
filtered_df = pd.concat([filtered_df, nat_rows], ignore_index=True)

# Ensure 'Code' column values are treated as strings
filtered_df['HCPCS or CPT Code'] = filtered_df['HCPCS or CPT Code'].astype(str)

# Function to create the 'Modifier(s)' column
def extract_modifiers(code):
    if len(code) <= 5:
        return ""  # Blank if only 5 characters
    elif len(code) == 7:
        return code[5:7]  # Only last 2 characters if there are 7
    elif len(code) > 7:
        return f"{code[5:7]},{code[7:9]}"  # Characters 6-7 followed by a comma and next 2 characters
    else:
        return None  # Handle unexpected cases

# Apply the function to create 'Modifier(s)' column
filtered_df['Modifier'] = filtered_df['HCPCS or CPT Code'].apply(extract_modifiers)

# Remove the modifiers from the 'Code' column (retain only the first 5 characters)
filtered_df['HCPCS or CPT Code'] = filtered_df['HCPCS or CPT Code'].str[:5]

# Drop 'Type ID' and 'EffectiveDateTime' columns
filtered_df = filtered_df.drop(columns=['EffectiveDateTime (DBarProcAltCodeEffectDates)'])

# Sort by 'Account Number' column in ascending order
sorted_df = filtered_df.sort_values(by='Encounter ID', ascending=True)

# Save the final DataFrame to a CSV file
sorted_df.to_csv("Charge Detail File Month.csv", index=False)

<ipython-input-2-e777fbde54c2>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['EffectiveDateTime (DBarProcAltCodeEffectDates)'] = pd.to_datetime(df['EffectiveDateTime (DBarProcAltCodeEffectDates)'])


In [ ]:
# Charge Detail File Final Step
import pandas as pd

# Load the data
df = pd.read_csv("Charge Detail File Month.csv")

# Sort by 'Encounter ID' and 'CDM Item Code' in ascending order
df = df.sort_values(by=['Encounter ID', 'CDM Item Code'])

# Group by 'CDM Item Code' and filter out rows where both CPT-4 and HCPCS exist
def filter_rows(group):
    if len(group) > 1 and 'CPT-4' in group['Type ID'].values and 'HCPCS' in group['Type ID'].values:
        # If both CPT-4 and HCPCS exist for the same CDM Item Code, keep the CPT-4 row
        return group[group['Type ID'] == 'CPT-4']
    else:
        # Otherwise, keep all rows
        return group

# Apply the filter to each group
filtered_df = df.groupby('CDM Item Code').apply(filter_rows).reset_index(drop=True)

# Drop the 'Type ID' column
filtered_df = filtered_df.drop(columns=['Type ID'])

# Select the desired columns
final_df = filtered_df[['Encounter ID', 'CDM Item Code', 'Service Date', 'Revenue Code', 'Units', 'Total Charge', 'Cost center/Department', 'HCPCS or CPT Code', 'Modifier']]

# Sort the final output by 'Encounter ID'
final_df = final_df.sort_values(by=['Encounter ID'])

# Save the result to a new CSV file
final_df.to_csv("Charge Detail File Month Sample.csv", index=False)

<ipython-input-3-56f79cdbac37>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = df.groupby('CDM Item Code').apply(filter_rows).reset_index(drop=True)


In [ ]:
import pandas as pd

# Load CSV files
claims = pd.read_csv('Claim Summary Month Sample.csv') # Columns: Encounter ID, Total Charges
charges = pd.read_csv('Charge Detail File Month Sample.csv') # Columns: Encounter ID, Total Charge

# Rename columns for clarity and to avoid merge conflicts
claims = claims.rename(columns={'Total Charges': 'total_charge_claims'})
charges = charges.rename(columns={'Total Charge': 'charge'})

# Sum charges per Encounter ID in the detail file
charge_sum = charges.groupby('Encounter ID')['charge'].sum().reset_index()

# Merge the two datasets
merged = pd.merge(
    claims,
    charge_sum,
    on='Encounter ID',
    how='outer', # Checks for IDs missing in either file
    suffixes=('_claims', '_charge')
)

# Flag discrepancies
merged['discrepancy'] = False

# Case 1: Totals don’t match (allow $0.01 tolerance for floating-point errors)
mask = (
    merged['total_charge_claims'].round(2) !=
    merged['charge'].round(2)
)
merged.loc[mask, 'discrepancy'] = True

# Case 2: Missing Encounter IDs in either file
merged['discrepancy'] = merged['discrepancy'] | merged[['total_charge_claims', 'charge']].isna().any(axis=1)

# Filter and display discrepancies
discrepancies = merged[merged['discrepancy']][['Encounter ID', 'total_charge_claims', 'charge']]

# Print results
print("Discrepancies:")
print(discrepancies.to_string(index=False))

# Optional: Save to CSV
discrepancies.to_csv('discrepancy_report.csv', index=False)


Discrepancies:
Encounter ID  total_charge_claims  charge
    VAT62552               120.01  120.00
    VAU14852               763.01  763.00
    VAU22472               400.01  400.00
    VAU24202               873.01  873.00
    VAU25025               210.01  210.00
    VAU25027               610.01  610.00
    VAU31310               763.01  763.00
    VAU34029              1865.02 1865.01
    VAU42238               447.00     NaN
    VAU42241               302.02  302.00
    VAU42245               453.01  453.00
    VAU42258               453.01  453.00
    VAU42260               453.01  453.00
    VAU47509               302.01  302.00
    VAU48733               610.01  610.00
    VAU48955               102.01  102.00
    VAU53926               692.00     NaN
    VAU57977               323.01  323.00
    VAU66284               127.21  127.20
    VAU73094              4071.20 4070.40
    VAU77846              4192.96 4192.95
    VAU82284               120.01  120.00
    VAU94939       